In [1]:
import tempfile
import os
import uuid
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.vectorstores import FAISS
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chat_models.gigachat import GigaChat
from langchain_community.embeddings import HuggingFaceEmbeddings
import logging 
import PyPDF2
import numpy as np
import os
os.environ['CURL_CA_BUNDLE'] = ''
import warnings
warnings.filterwarnings("ignore")

In [36]:
llm = GigaChat(credentials='ZjYyNjZkYzAtYzEwYS00NDZjLWIyMTMtMzc3N2U1ZTNmNTI2OmZlMGE3ZGVmLWFlYjEtNGNiZC05NzAxLWZhMTNkMDU4NDg5Zg==', 
               verify_ssl_certs=False, max_tokens=10)
def get_embeddings():
    model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
    model_kwargs = {'device': 'cpu'}
    # model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': False}
    return HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
        )

In [19]:
%%time
from langchain_community.document_loaders import Docx2txtLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import PDFMinerLoader
)
documents = []
for file in os.listdir('files'):
    if file.endswith('.pdf'):
        try:
            pdf_path = './files/' + file
            loader = PDFMinerLoader(pdf_path)
            documents.extend(loader.load())
            print('Да')
        except:
            print('Нет')
    elif file.endswith('.docx') or file.endswith('.doc'):
        doc_path = './files/' + file
        loader = Docx2txtLoader(doc_path)
        documents.extend(loader.load())
    elif file.endswith('.txt'):
        text_path = './files/' + file
        loader = TextLoader(text_path, encoding='utf-8')
        documents.extend(loader.load())

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200,)
splits = text_splitter.split_documents(documents)
embedding = get_embeddings()
vectorstore = FAISS.from_documents(documents=splits, embedding=embedding)
retriever = vectorstore.as_retriever(search_kwargs={"k":3})

SyntaxError: unmatched ')' (<unknown>, line 4)

In [16]:
from langchain_community.document_loaders import PyMuPDFLoader
documents = []
for file in os.listdir('files'):
    if file.endswith('.pdf'):
        try:
            pdf_path = './files/' + file
            loader = PyMuPDFLoader(pdf_path)
            documents.extend(loader.load())
            print('Да')
        except:
            print('Нет')

Да
Да
Да
Да
Да
Да
Да
Да
Да
Нет
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Нет
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Нет
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Нет
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Нет
Да
Да
Да
Да
Да
Нет
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Нет
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Нет
Да
Да
Да
Нет
Да
Нет
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
MuPDF error: library error: zlib error: invalid stored block lengths

Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да
Да


In [20]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=900, chunk_overlap=300,)
splits = text_splitter.split_documents(documents)
embedding = get_embeddings()
vectorstore = FAISS.from_documents(documents=splits, embedding=embedding)
vectorstore.save_local(folder_path='db2/')
retriever = vectorstore.as_retriever(search_kwargs={"k":3})

In [37]:
contextualize_q_system_prompt = (
       "Запоминай историю чата и последний вопрос пользвоателя"
    "Сначала пытайся ответить на вопрос, основываясь на истории чата"
    "сформулируй отдельный вопрос без истории чата"
    "Не отвечай на этот вопрос, просто перефразируй его, если это необходимо, в противном случае верни как есть"
    )
contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextualize_q_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )
history_aware_retriever = create_history_aware_retriever(
        llm, retriever, contextualize_q_prompt
    )

    ### Answer question ###
system_prompt = (
       ''' 
    Ты подробно и структурировоно отвечаешь на вопрос на основе загруженных статей об искуственном интелекте.
    Предлагай пользователю задать дополнительные похожие вопросы. Предлагайте контекстную и справочную информацию,
    основываясь на метаданнтых каждого документа из контекста.
    Если ответа на вопрос нет в контексте, предложи вопрос, ответ на который есть в контексте
    Контекст: {context}
    Ответ:'''
    )
qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

    ### Statefully manage chat history ###
store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
        if session_id not in store:
            store[session_id] = ChatMessageHistory()
        return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
        rag_chain,
        get_session_history,
        input_messages_key="input",
        history_messages_key="chat_history",
        output_messages_key="answer",
    )


In [38]:
conversational_rag_chain.invoke({'input': 'Пhивет'}, config={
                        "configurable": {"session_id": 'abc123'}
                    })

Giga generation stopped with reason: length


{'input': 'Пhивет',
 'chat_history': [],
 'context': [Document(metadata={'source': './files/-7918469119355088835.pdf', 'file_path': './files/-7918469119355088835.pdf', 'page': 16, 'total_pages': 34, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Microsoft® PowerPoint® 2016', 'producer': 'Microsoft® PowerPoint® 2016', 'creationDate': "D:20240605093603+03'00'", 'modDate': "D:20240605093603+03'00'", 'trapped': ''}, page_content='\uf0d2'),
  Document(metadata={'source': './files/5804862233080142600.pdf', 'file_path': './files/5804862233080142600.pdf', 'page': 4, 'total_pages': 51, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'macOS Version 14.4.1 (Build 23E224) Quartz PDFContext', 'creationDate': "D:20240608114311Z00'00'", 'modDate': "D:20240608114311Z00'00'", 'trapped': ''}, page_content='CURRENT AI USAGE'),
  Document(metadata={'source': './files/2985244266833291753.pdf', 'file_path':

In [42]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context


# generator with openai models
generator_llm = GigaChat(credentials='ZjYyNjZkYzAtYzEwYS00NDZjLWIyMTMtMzc3N2U1ZTNmNTI2OmZlMGE3ZGVmLWFlYjEtNGNiZC05NzAxLWZhMTNkMDU4NDg5Zg==', 
               verify_ssl_certs=False, max_tokens=20)
critic_llm = GigaChat(credentials='ZjYyNjZkYzAtYzEwYS00NDZjLWIyMTMtMzc3N2U1ZTNmNTI2OmZlMGE3ZGVmLWFlYjEtNGNiZC05NzAxLWZhMTNkMDU4NDg5Zg==', 
               verify_ssl_certs=False, max_tokens=20)
embeddings = get_embeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

# generate testset
testset = generator.generate_with_langchain_docs(documents[:50], test_size=5, distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25})

embedding nodes:   0%|          | 0/322 [00:00<?, ?it/s]Giga generation stopped with reason: length
Giga generation stopped with reason: length
embedding nodes:   2%|▏         | 6/322 [00:02<01:27,  3.60it/s]Giga generation stopped with reason: length
Giga generation stopped with reason: length
Giga generation stopped with reason: length
embedding nodes:   4%|▍         | 13/322 [00:04<01:19,  3.88it/s]Giga generation stopped with reason: length
Giga generation stopped with reason: length
Giga generation stopped with reason: length
Giga generation stopped with reason: length
Giga generation stopped with reason: length
Giga generation stopped with reason: length
Giga generation stopped with reason: length
embedding nodes:   5%|▌         | 17/322 [00:04<01:05,  4.65it/s]Giga generation stopped with reason: length
Giga generation stopped with reason: length
Giga generation stopped with reason: length
Giga generation stopped with reason: length
Giga generation stopped with reason: length
Gi

KeyboardInterrupt: 

Giga generation stopped with reason: length
Giga generation stopped with reason: length
Giga generation stopped with reason: length
Giga generation stopped with reason: length
Giga generation stopped with reason: length
Giga generation stopped with reason: length
Giga generation stopped with reason: length
Giga generation stopped with reason: length
Giga generation stopped with reason: length
Giga generation stopped with reason: length
Giga generation stopped with reason: length
Giga generation stopped with reason: length
Giga generation stopped with reason: length
Giga generation stopped with reason: length
Giga generation stopped with reason: length
Giga generation stopped with reason: length
Giga generation stopped with reason: length
Giga generation stopped with reason: length
Giga generation stopped with reason: length
Giga generation stopped with reason: length
Giga generation stopped with reason: length
Giga generation stopped with reason: length
Giga generation stopped with rea

In [40]:
testset

NameError: name 'testset' is not defined